In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "947074e8-9ff9-494a-8259-d1a98416da35",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "╒═══════════╤════════════╤═══════════════╤═════════════════════════════════════════════════╤═══════════╕\n",
      "│ Keyword   │ Category   │   Match Score │ Title                                           │ URL       │\n",
      "╞═══════════╪════════════╪═══════════════╪═════════════════════════════════════════════════╪═══════════╡\n",
      "│ SEO       │ SEO        │          0.82 │ The Complete SEO Glossary                       │ article-6 │\n",
      "├───────────┼────────────┼───────────────┼─────────────────────────────────────────────────┼───────────┤\n",
      "│ RFP       │ RFP        │          0.84 │ 10 RFP Questions for Your Business | Marketwake │ article-1 │\n",
      "╘═══════════╧════════════╧═══════════════╧═════════════════════════════════════════════════╧═══════════╛\n"
     ]
    }
   ],
   "source": [
    "from openai import OpenAI\n",
    "from pinecone import Pinecone\n",
    "from IPython.display import clear_output\n",
    "from tabulate import tabulate  # Import tabulate for table formatting\n",
    "\n",
    "openai_client = OpenAI(api_key='YOUR_API_KEY')  # Instantiate OpenAI client\n",
    "pinecone = Pinecone(api_key='YOUR_API_KEY')\n",
    "\n",
    "index_name = \"internal-link-tool\"\n",
    "index = pinecone.Index(index_name)\n",
    "\n",
    "\n",
    "# Function to generate embeddings using OpenAI's API\n",
    "def generate_embeddings(text):\n",
    "    \"\"\"\n",
    "    Generates an embedding for a given text using OpenAI's API.\n",
    "\n",
    "    \"\"\"\n",
    "    try:\n",
    "        if not text or not isinstance(text, str):\n",
    "            raise ValueError(\"Input text must be a non-empty string.\")\n",
    "\n",
    "        result = openai_client.embeddings.create(\n",
    "            input=text,\n",
    "            model=\"text-embedding-ada-002\"\n",
    "        )\n",
    "\n",
    "        # Debugging: Print the response to understand its structure\n",
    "        clear_output(wait=True)\n",
    "        #print(\"API Response:\", result)\n",
    "\n",
    "        if hasattr(result, 'data') and len(result.data) > 0:\n",
    "            return result.data[0].embedding\n",
    "        else:\n",
    "            raise ValueError(\"Invalid response from the OpenAI API. No data returned.\")\n",
    "\n",
    "    except ValueError as ve:\n",
    "        print(f\"ValueError: {ve}\")\n",
    "        return None\n",
    "\n",
    "    except Exception as e:\n",
    "        print(f\"An error occurred while generating embeddings: {e}\")\n",
    "        return None\n",
    "\n",
    "# Function to query the Pinecone index with keywords and metadata\n",
    "def match_keywords_to_index(keywords):\n",
    "    \"\"\"\n",
    "    Matches a list of keywords to the closest article in the Pinecone index, filtering by metadata dynamically.\n",
    "    \"\"\"\n",
    "    results = []\n",
    "\n",
    "    for keyword_pair in keywords:\n",
    "        try:\n",
    "            clear_output(wait=True)\n",
    "            # Extract the keyword and category from the sub-array\n",
    "            keyword = keyword_pair[0]\n",
    "            category = keyword_pair[1]\n",
    "\n",
    "            # Generate embedding for the current keyword\n",
    "            vector = generate_embeddings(keyword)\n",
    "            if vector is None:\n",
    "                print(f\"Skipping keyword '{keyword}' due to embedding error.\")\n",
    "                continue\n",
    "\n",
    "            # Query the Pinecone index for the closest vector with metadata filter\n",
    "            query_results = index.query(\n",
    "                vector=vector,  # The embedding of the keyword\n",
    "                top_k=1,  # Retrieve only the closest match\n",
    "                include_metadata=True,  # Include metadata in the results\n",
    "                filter={\"category\": category}  # Filter results by metadata category dynamically\n",
    "            )\n",
    "\n",
    "            # Store the closest match\n",
    "            if query_results['matches']:\n",
    "                closest_match = query_results['matches'][0]\n",
    "                results.append({\n",
    "                    'Keyword': keyword,  # The searched keyword\n",
    "                    'Category': category,  # The category used for filtering\n",
    "                    'Match Score': f\"{closest_match['score']:.2f}\",  # Similarity score (formatted to 2 decimal places)\n",
    "                    'Title': closest_match['metadata'].get('title', 'N/A'),  # Title of the article\n",
    "                    'URL': closest_match['id']  # Using 'id' as the URL\n",
    "                })\n",
    "            else:\n",
    "                results.append({\n",
    "                    'Keyword': keyword,\n",
    "                    'Category': category,\n",
    "                    'Match Score': 'N/A',\n",
    "                    'Title': 'No match found',\n",
    "                    'URL': 'N/A'\n",
    "                })\n",
    "\n",
    "        except Exception as e:\n",
    "            clear_output(wait=True)\n",
    "            print(f\"Error processing keyword '{keyword}' with category '{category}': {e}\")\n",
    "            results.append({\n",
    "                'Keyword': keyword,\n",
    "                'Category': category,\n",
    "                'Match Score': 'Error',\n",
    "                'Title': 'Error occurred',\n",
    "                'URL': 'N/A'\n",
    "            })\n",
    "\n",
    "    return results\n",
    "\n",
    "# SEARCH KEYWORDS HERE (KEYWORD, CATEGORY)\n",
    "keywords = [[\"SEO\", \"SEO\"], [\"RFP\",\"RFP\"]]\n",
    "matches = match_keywords_to_index(keywords)\n",
    "\n",
    "print(tabulate(matches, headers=\"keys\", tablefmt=\"fancy_grid\"))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9bf83754-99d2-4df9-b1ca-23813d89e211",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
